In [ ]:
import os
import json
import sys

In [ ]:
# Importamos google drive donde se almacenan los códigos y archivos
from google.colab import drive
drive.mount('/content/drive')

# Código

In [ ]:
# 1. Ruta de trabajo
WORK_DIR = '/content/drive/MyDrive/PruebasSoftware2026/A6.2_ArchivosApoyo'

# 2. Creación del directorio
if not os.path.exists(WORK_DIR):
    os.makedirs(WORK_DIR)
    print(f"Directorio creado: {WORK_DIR}")
else:
    print(f"Directorio existente: {WORK_DIR}")

# 3. Cambiar el directorio de trabajo
os.chdir(WORK_DIR)
print(f"Directorio de trabajo actual: {os.getcwd()}")

`hotel_system.py`

In [ ]:
%%writefile hotel_system.py
"""
Sistema para gestionar Hoteles, Clientes y Reservaciones.
Persiste los datos en archivos JSON en un directorio específico.
"""

import os
import json

# Definir ruta base
BASE_DIR = '/content/drive/MyDrive/PruebasSoftware2026/A6.2_ArchivosApoyo'


class FileManager:
    """Clase auxiliar para manejar operaciones de archivos."""

    @staticmethod
    def get_filepath(filename):
        """Devuelve la ruta absoluta de un archivo."""
        return os.path.join(BASE_DIR, filename)

    @staticmethod
    def load_data(filename):
        """Carga los datos de un archivo JSON."""
        filepath = FileManager.get_filepath(filename)
        if not os.path.exists(filepath):
            return []
        try:
            with open(filepath, 'r', encoding='utf-8') as file:
                return json.load(file)
        except (json.JSONDecodeError, IOError):
            print(f"Error: No se pudo leer '{filepath}'. Devolviendo lista vacía.")
            return []

    @staticmethod
    def save_data(filename, data):
        """Guarda los datos en un archivo JSON."""
        filepath = FileManager.get_filepath(filename)
        try:
            with open(filepath, 'w', encoding='utf-8') as file:
                json.dump(data, file, indent=4)
        except IOError:
            print(f"Error: No se pudo guardar en '{filepath}'.")


class Hotel:
    """Clase para gestionar las entidades de Hotel."""
    FILE = 'hotels.json'

    def __init__(self, hotel_id, name, location, rooms):
        self.hotel_id = hotel_id
        self.name = name
        self.location = location
        self.rooms = rooms

    def to_dict(self):
        """Devuelve la representación en diccionario del hotel."""
        return self.__dict__

    @classmethod
    def create_hotel(cls, hotel_id, name, location, rooms):
        """Crea un nuevo hotel y lo guarda en el archivo."""
        hotels = FileManager.load_data(cls.FILE)
        if any(h['hotel_id'] == hotel_id for h in hotels):
            print(f"Error: El ID del hotel {hotel_id} ya existe.")
            return
        new_hotel = cls(hotel_id, name, location, rooms)
        hotels.append(new_hotel.to_dict())
        FileManager.save_data(cls.FILE, hotels)
        print(f"Hotel '{name}' creado exitosamente.")

    @classmethod
    def delete_hotel(cls, hotel_id):
        """Elimina un hotel por su ID."""
        hotels = FileManager.load_data(cls.FILE)
        new_list = [h for h in hotels if h['hotel_id'] != hotel_id]
        if len(hotels) == len(new_list):
            print(f"Error: ID del hotel {hotel_id} no encontrado.")
        else:
            FileManager.save_data(cls.FILE, new_list)
            print(f"ID del hotel {hotel_id} eliminado.")

    @classmethod
    def display_hotel_info(cls, hotel_id):
        """Muestra la información de un hotel específico."""
        hotels = FileManager.load_data(cls.FILE)
        hotel = next((h for h in hotels if h['hotel_id'] == hotel_id), None)
        if hotel:
            print(f"Información del Hotel: {hotel}")
            return hotel
        print(f"Error: ID del hotel {hotel_id} no encontrado.")
        return None

    @classmethod
    def modify_hotel_info(cls, hotel_id, **kwargs):
        """Modifica los atributos de un hotel existente."""
        hotels = FileManager.load_data(cls.FILE)
        found = False
        for hotel in hotels:
            if hotel['hotel_id'] == hotel_id:
                hotel.update(kwargs)
                found = True
                break

        if found:
            FileManager.save_data(cls.FILE, hotels)
            print(f"ID del hotel {hotel_id} actualizado.")
        else:
            print(f"Error: ID del hotel {hotel_id} no encontrado.")

    @classmethod
    def reserve_room(cls, hotel_id):
        """Decrementa las habitaciones disponibles de un hotel."""
        hotels = FileManager.load_data(cls.FILE)
        for hotel in hotels:
            if hotel['hotel_id'] == hotel_id:
                if hotel['rooms'] > 0:
                    hotel['rooms'] -= 1
                    FileManager.save_data(cls.FILE, hotels)
                    return True
                print(f"Error: No hay habitaciones disponibles en el Hotel {hotel_id}.")
                return False
        print(f"Error: ID del hotel {hotel_id} no encontrado.")
        return False

    @classmethod
    def cancel_reservation(cls, hotel_id):
        """Incrementa las habitaciones disponibles de un hotel."""
        hotels = FileManager.load_data(cls.FILE)
        for hotel in hotels:
            if hotel['hotel_id'] == hotel_id:
                hotel['rooms'] += 1
                FileManager.save_data(cls.FILE, hotels)
                return
        print(f"Error: ID del hotel {hotel_id} no encontrado.")


class Customer:
    """Clase para gestionar las entidades de Cliente."""
    FILE = 'customers.json'

    def __init__(self, customer_id, name, email):
        self.customer_id = customer_id
        self.name = name
        self.email = email

    def to_dict(self):
        """Devuelve la representación en diccionario del cliente."""
        return self.__dict__

    @classmethod
    def create_customer(cls, customer_id, name, email):
        """Crea un nuevo cliente."""
        customers = FileManager.load_data(cls.FILE)
        if any(c['customer_id'] == customer_id for c in customers):
            print(f"Error: El ID del cliente {customer_id} ya existe.")
            return
        new_cust = cls(customer_id, name, email)
        customers.append(new_cust.to_dict())
        FileManager.save_data(cls.FILE, customers)
        print(f"Cliente '{name}' creado exitosamente.")

    @classmethod
    def delete_customer(cls, customer_id):
        """Elimina un cliente."""
        customers = FileManager.load_data(cls.FILE)
        new_list = [c for c in customers if c['customer_id'] != customer_id]
        if len(customers) == len(new_list):
            print(f"Error: ID del cliente {customer_id} no encontrado.")
        else:
            FileManager.save_data(cls.FILE, new_list)
            print(f"ID del cliente {customer_id} eliminado.")

    @classmethod
    def display_customer_info(cls, customer_id):
        """Muestra la información del cliente."""
        customers = FileManager.load_data(cls.FILE)
        cust = next(
            (c for c in customers if c['customer_id'] == customer_id),
            None
        )
        if cust:
            print(f"Información del Cliente: {cust}")
            return cust
        print(f"Error: ID del cliente {customer_id} no encontrado.")
        return None

    @classmethod
    def modify_customer_info(cls, customer_id, **kwargs):
        """Modifica la información del cliente."""
        customers = FileManager.load_data(cls.FILE)
        found = False
        for cust in customers:
            if cust['customer_id'] == customer_id:
                cust.update(kwargs)
                found = True
                break

        if found:
            FileManager.save_data(cls.FILE, customers)
            print(f"ID del cliente {customer_id} actualizado.")
        else:
            print(f"Error: ID del cliente {customer_id} no encontrado.")


class Reservation:
    """Clase para gestionar las Reservaciones."""
    FILE = 'reservations.json'

    def __init__(self, reservation_id, customer_id, hotel_id):
        self.reservation_id = reservation_id
        self.customer_id = customer_id
        self.hotel_id = hotel_id

    def to_dict(self):
        """Devuelve la representación en diccionario de la reservación."""
        return self.__dict__

    @classmethod
    def create_reservation(cls, reservation_id, customer_id, hotel_id):
        """Crea una reservación si el hotel y el cliente existen."""
        # Verificar que el cliente existe
        cust = Customer.display_customer_info(customer_id)
        if not cust:
            print("Fallo en la reservación: Cliente no encontrado.")
            return

        # Verificar que el hotel existe y reservar habitación
        if not Hotel.reserve_room(hotel_id):
            print("Fallo en la reservación: Hotel no encontrado o sin habitaciones.")
            return

        reservations = FileManager.load_data(cls.FILE)
        new_res = cls(reservation_id, customer_id, hotel_id)
        reservations.append(new_res.to_dict())
        FileManager.save_data(cls.FILE, reservations)
        print(f"Reservación {reservation_id} creada exitosamente.")

    @classmethod
    def cancel_reservation(cls, reservation_id):
        """Cancela una reservación y libera la habitación."""
        reservations = FileManager.load_data(cls.FILE)
        res = next(
            (r for r in reservations if r['reservation_id'] == reservation_id),
            None
        )
        if not res:
            print(f"Error: Reservación {reservation_id} no encontrada.")
            return

        # Liberar la habitación en el hotel
        Hotel.cancel_reservation(res['hotel_id'])

        # Eliminar reservación
        new_list = [
            r for r in reservations
            if r['reservation_id'] != reservation_id
        ]
        FileManager.save_data(cls.FILE, new_list)
        print(f"Reservación {reservation_id} cancelada.")

# Generación de datos insumo para el programa.

In [ ]:
# Ruta base
BASE_DIR = '/content/drive/MyDrive/PruebasSoftware2026/A6.2_ArchivosApoyo'

# Datos iniciales
hotels_data = [
    {"hotel_id": "H1", "name": "Hotel California", "location": "USA", "rooms": 10},
    {"hotel_id": "H2", "name": "Grand Hotel", "location": "Spain", "rooms": 5}
]

customers_data = [
    {"customer_id": "C1", "name": "John Doe", "email": "john@example.com"},
    {"customer_id": "C2", "name": "Jane Smith", "email": "jane@example.com"}
]

# Función auxiliar para escribir en la ruta correcta
def write_json(filename, data):
    filepath = os.path.join(BASE_DIR, filename)
    with open(filepath, 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=4)
    print(f"Archivo generado: {filepath}")

# Generación de archivos
write_json('hotels.json', hotels_data)
write_json('customers.json', customers_data)
write_json('reservations.json', [])

Pruebas sobre el código generado

In [ ]:
WORK_DIR = '/content/drive/MyDrive/PruebasSoftware2026/A6.2_ArchivosApoyo'
if WORK_DIR not in sys.path:
    sys.path.append(WORK_DIR)
os.chdir(WORK_DIR)

# 1. Importar tus clases desde el archivo 'hotel_system.py'
try:
    from hotel_system import Hotel, Customer, Reservation
    print("Módulo 'hotel_system' importado correctamente.\n")
except ImportError as e:
    print(f"Error al importar: {e}")


# --- BLOQUE DE PRUEBAS de métodos PUT/GET ---

print("--- 1. PRUEBA DE ESCRITURA ---")
# Crear un Hotel Nuevo
print(">> Registrando 'Hotel Pruebas'...")
Hotel.create_hotel("H-TEST-01", "Hotel Python", "Cancún", 2)

# Crear un Cliente Nuevo
print(">> Registrando Cliente 'Usuario Test'...")
Customer.create_customer("C-TEST-01", "Usuario Test", "test@itesm.mx")

# Crear una Reservación (Esto debería restar 1 habitación al hotel)
print(">> Generando Reservación...")
Reservation.create_reservation("R-TEST-01", "C-TEST-01", "H-TEST-01")


print("\n--- 2. PRUEBA DE LECTURA Y ESTADO ---")
# Verificar info del Hotel
hotel_info = Hotel.display_hotel_info("H-TEST-01")

# Verificar info del Cliente
cust_info = Customer.display_customer_info("C-TEST-01")


print("\n--- 3. VERIFICACIÓN DE PERSISTENCIA (JSON) ---")
# Leeremos directamente el archivo JSON para asegurar que se guardó en disco
def leer_json_raw(nombre_archivo):
    with open(nombre_archivo, 'r') as f:
        data = json.load(f)
        # Filtramos solo los datos de prueba para mostrar
        items = [d for d in data if "TEST" in str(d.values())]
        print(f"Contenido en {nombre_archivo} (filtrado):")
        print(json.dumps(items, indent=2))

leer_json_raw('hotels.json')
leer_json_raw('reservations.json')

print("\n--- PRUEBAS FINALIZADAS ---")